# CIFAR-10 Classification
> Performance analysis for CIFAR-10 Classification on all hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [CIFAR-10,Rooflines,Performance Prediction]
- image: images/cifar_logo.png

In [1]:
#hide
#from altair.vegalite.v4.api import Selection
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

pd.options.display.max_rows = 500 # this will set limit of rows to 500
pd.options.display.max_columns = 500 # this will set limit of columns to 500

pd.options.display.float_format = '{:20,.4f}'.format

csv_path = "./data/cleaned_csv/backup.csv"

# Theretical Analysis of CIFAR-10

### Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [2]:
#hide_input

#first process the following csv's to get clean ready-to-plot csv's
%run scripts/script_load_save_data.py
clean_csv_rooflines(path_topologies='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/topology_details.csv',
                    path_hardware='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/peakPerfBandHardPlatf.csv')

#Now get the cleaned csv, and plot it as a Roofline
%run scripts/altair_plots.py
rooflines(dataframe = pd.read_csv("data/cleaned_csv/rooflines_hardware_neural_networks.csv"), 
          neural_network = 'cifar')

alt.LayerChart(...)

### Performance Prediction

The following heatmap shows the theoretical performance for the listed hardware platforms for CIFAR-10 classification. The metric used for the theoretical performance is input/second.
We observe that prunning along with quantization outputs some of the best performance results.

In [3]:
#hide_input
path_csv = 'data/performance_predictions_imagenet_mnist_cifar_2.csv'
    ## Reading csv file and converting data to (Neural network, Platform, Value)
df = pd.read_csv(path_csv)

df1 = pd.DataFrame()
columns = (df.loc[:, df.columns!='hardw']).columns #select all columns except first
for column in columns:
    df_=pd.melt(df, id_vars=['hardw'], value_vars=column) #melt df1 into a df1 of 2 columns
    df1=pd.concat([df1,df_])
df1.columns= ['y','x','values'] #setting new column names
#replace 0s for NaN values because with 0s the grid doesn't show up
df1['values'] = df1['values'].replace({ 0.0:np.nan})
df_cifar10 = dataframe_contains(input_df=df1, column='x', value='CNV')
df_cifar10.to_csv('c:/Users/alinav/Documents/GitHub/QutibenchWeb/_notebooks/data/cleaned_csv/performance_prediction_cifar10.csv', index = False)

heatmap(df_cifar10, 'pink', 'Performance prediction for CIFAR-10')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for CIFAR-10

In this table, within the rows, we show the type of hardware platforms that we used for this task (for example FPGA or GPU) and then more specifically the exact name of the different hardware platforms. For each hardware platform, we list the sweep of specific deployment parameters (batch sizes, operating modes etc) that were used for the experimentation in separate columns. In the columns, we show CNN topologies. When a CNN topology was implemented on a given hardware platform, we show in the corresponding cell the precisions (quantization information) and the channel pruning scale. Otherwise, “na” indicates that the topology wasn’t executed on this specific hardware platform. Many combinations between topology and hardware platform are not supported by the vendors dedicated software environments. INTx depicts a fixed point integer representation with x bits. FPy represents a floating point representation with y bits, for example FP32 is singe precision floating point. Table follows below.

In [4]:
#hide
%run scripts/altair_plots.py   #get table with the experiments overview
print(pd.read_csv('data/overview_experiments_cifar10.csv').to_markdown())

|    | Hardware   | Platform         | CNV                              | Batch/Stream/Thread                  |
|---:|:-----------|:-----------------|:---------------------------------|:-------------------------------------|
|  0 | FPGA       | ZCU102-DPU       | na                               | [1,2,3,4,5,6,7,8]                    |
|  1 | FPGA       | ZCU104-DPU       | na                               | [1,2,3,4,5,6,7,8]                    |
|  2 | FPGA       | Ultra96-DPU      | na                               | [1,2,3,4,5,6,7,8]                    |
|  3 | FPGA       | ZCU104-FINN      | [INT2,INT4]*[100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128,256,512,10000] |
|  4 | FPGA       | ZCU104-BISMO     | [INT2,INT4]*[100%,50%,25%,12.5%] | [2,4,8,16,32,64,128]                 |
|  5 | GPU        | TX2-maxn         | [FP16,FP32]*[100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128]               |
|  6 | GPU        | TX2-maxp         | [FP16,FP32]*[100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128]

In [5]:
#hide_input
%run scripts/script_tables.py 
#get table with the experiments overview
dataframes = csv_to_dataframe_multiindex(['data/overview_experiments_cifar10_.csv'])
for dataframe in dataframes:   
       display(HTML(dataframe.to_html(index=False)))

In [6]:
a=2
str(a)

'2'

In [7]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "GPU" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
maxp_df["hw_datatype_prun"] = maxp_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.PruningFactor]), axis=1)
cnv_df = maxp_df[(maxp_df["NN_Topology"] == "CNV") & maxp_df['lat-comp'].notna() & maxp_df["top1 [%]"].notna()]
cnv_df["hw_datatype_prun"] = cnv_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.PruningFactor]), axis=1)
cnv_df["PruningFactor"] = cnv_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(cnv_df, "lat-comp", "NN_Topology");
cnv_df["datatype_model"] = cnv_df.Datatype + '_' + cnv_df.HWType
cnv_df["tag"] = cnv_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.NN_Topology, str(r.PruningFactor)]), axis=1)
cnv_df.rename(columns={"top1 [%]": "top1"}, inplace=True)

#fill GOPS gap
cnv_df['GOPS'] = cnv_df.apply(lambda r: 0.469450 if r.PruningFactor == 100.00 else 
                                          (0.118923636 if r.PruningFactor == 50.00 else 
                                           (0.030511732 if r.PruningFactor == 25.00 else 0.008018676 )) , axis=1)

#fill in tp-system and tp-cmp
cnv_df['tp-system'] = cnv_df['fps-system'] * cnv_df['GOPS']
cnv_df['tp-comp'] = cnv_df['fps-comp'] * cnv_df['GOPS']
cnv_df['GOPS'] = cnv_df['GOPS'] * cnv_df['batch/thread/stream']
#cnv_df

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\alinav\Documents\GitHub\QutibenchWeb\_notebooks\scripts\altair_plots.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [8]:
#hide
figa_df = cnv_df[(cnv_df["HWType"].isin(["NCS", "ZCU104-Bismo", "U96-Quadcore A53"]))]
figb_df = cnv_df[(cnv_df["HWType"].isin(["GPU", "ZCU104-FINN", "U96-Quadcore A53"]))]

### Line Plot

In [9]:
#hide_input
fig25s = []
fig25_dfs = [figa_df, figb_df]
for df in fig25_dfs:
    sel = alt.selection_multi(fields=["hw_datatype_prun"], bind="legend")
    fig25_dot = alt.Chart(df).mark_point().encode(
        x='lat-comp',
        y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
        color=select_color(sel, 'hw_datatype_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )
    fig25_line = alt.Chart(df).mark_line().encode(
        x='lat-comp',
        y='fps-comp',
        color=select_color(sel, 'hw_datatype_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )

    fig = (fig25_dot+fig25_line).properties(
        title="Latency versus Performance for Pruned and Quantized CNV Variants",
        width=W/len(fig25_dfs),
        height=H,
    ).add_selection(sel).interactive()
    
    fig25s.append(fig)
    
alt.hconcat(*fig25s)

alt.HConcatChart(...)

### Boxplots

In [10]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=cnv_df, 
        xaxis='PruningFactor', 
        yaxis="lat-comp", 
        color_col= 'PruningFactor', 
        facet_column='datatype_model', 
        title="Latency by Hardware/Framework and Pruning for CIFAR-10 Classification")

alt.FacetChart(...)

In [11]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=cnv_df, 
        xaxis='PruningFactor', 
        yaxis="fps-comp", 
        color_col= 'PruningFactor', 
        facet_column='datatype_model', 
        title="Throughput by Hardware/Framework and Pruning for  CIFAR-10 Classification")

alt.FacetChart(...)

In [12]:
#hide
# First delete datapoints that are only one value, so the plot doesnt appear empty
cnv_df_tmp = cnv_df
cnv_df_tmp = delete_unique_values(df= cnv_df_tmp, 
                                 col_a= 'HWType', 
                                 col_b='Full_Pwr_W' )

C:\Users\alinav\Documents\GitHub\QutibenchWeb\_notebooks\scripts\altair_plots.py:471: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count'] = df.groupby(col_a)[col_a].transform('count')


In [13]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=cnv_df_tmp, 
        xaxis='PruningFactor', 
        yaxis="Full_Pwr_W", 
        color_col= 'PruningFactor', 
        facet_column='datatype_model', 
        title="Power Consumption by Hardware/Framework and Pruning CIFAR-10 Classification")

alt.FacetChart(...)

### Pareto Graphs

The following pareto graph presents the accuracy versus performance in fps for all the Hardware Platforms across different Pruning and Quantization configurations. This provides insights into accuracy-based comparisons.

In [14]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
pareto_graph(df= cnv_df, 
             groupcol= 'hw_datatype_prun',
             xcol = 'fps-comp', 
             ycol= 'top1',
             W= W, 
             H= H, 
             title= "CNV Classification Design Space: Accuracy versus Performance")

alt.LayerChart(...)

In [15]:
#hide
def replace_data_df(df_: pd.DataFrame(), column:str, list_tuples_data_to_replace: list )-> pd.DataFrame():
    """Method to replace a substring inside a cell inside a dataframe"""
    df = df_.copy()
    for j, k in list_tuples_data_to_replace:
        df[column] = df[column].str.replace(j, k)
    return df

In [16]:
#hide
""" This is to create a df to be joined with the theoretical df in 'Theoretical Analysis' to create the overlapped paretos

Steps
------
1. Create subset from imagenet that doesn't have the ResNet50 v15 measurements because it does not have accuracy measures
2. Create new hardware column that has hardware and operation mode, beware with NaNs
3. Create new 'hardw_datatype_net_prun' with hardware + datatype + netwrok + pruning
4. Create a suset of the dataframe with the above mentioned column and the corresponding ones
5. With groupby for col 'hardw_datatype_net_prun', for each unique value get the rows with biggest batch 
6. Add 'type column', reset the index from 'hardw_datatype_net_prun' to ints and save it
"""
#This is to get the pareto plot for biggest batch size
#   Create cnv_df_tmp from cnv_df
cnv_df_tmp = cnv_df.copy()
#   create hardw column to include: hardware + op_mode
cnv_df_tmp['hardw']= cnv_df_tmp['HWType'] + ('-' + cnv_df_tmp['Op mode']).fillna('')
#   create hardw_datatype_net_prun col with all those columns merged
cnv_df_tmp['hardw_datatype_net_prun'] = cnv_df_tmp.apply(lambda r: "_".join([r.hardw, r.Datatype,r.NN_Topology, str(r.PruningFactor)]), axis=1)
#   create a subset of the dataframe with only those columns
cnv_df_tmp = cnv_df_tmp[['hardw_datatype_net_prun', 'hardw','NN_Topology','fps-comp', 'top1','batch/thread/stream']]
#   Only get the points corresponding to the biggest batch
cnv_df_tmp = cnv_df_tmp.groupby('hardw_datatype_net_prun')[['batch/thread/stream', 'hardw','NN_Topology','fps-comp', 'top1']].max()
#   add type column
cnv_df_tmp['type'] = 'measured'
#   reset index to start being numeric 
cnv_df_tmp = cnv_df_tmp.reset_index()
#   change column names
cnv_df_tmp.columns = ['hardw_datatype_net_prun', 'batch/thread/stream', 'hardw', 'network', 'fps-comp', 'top1', 'type']
#   fix samll stuff in the df so things match with the other side
cnv_df_tmp = replace_data_df(df_=cnv_df_tmp, column='hardw_datatype_net_prun', list_tuples_data_to_replace=[("RN50", "ResNet50"),("MNv1", "MobileNetv1"),('GNv1','GoogLeNetv1'),('100.0','100'),('25.0','25') ,('50.0','50')])
cnv_df_tmp = replace_data_df(df_=cnv_df_tmp, column='network', list_tuples_data_to_replace=[("RN50", "ResNet50"),("MNv1", "MobileNetv1"),('GNv1','GoogLeNetv1')])
#delete unnecessary columns
cnv_df_tmp = cnv_df_tmp.drop(columns=['batch/thread/stream'])


#save it all
cnv_df_tmp.to_csv('data/cleaned_csv/pareto_data_cifar.csv', index = False)
#plot the pareto to confirm all is good
pareto_graph(df= cnv_df_tmp, 
             groupcol= 'hardw_datatype_net_prun', 
             xcol= 'fps-comp', 
             ycol= 'top1', 
             W= W, 
             H= H, 
             title= "CIFAR-10 Cassification Design Space: Accuracy versus Performance")
#cnv_df_tmp

alt.LayerChart(...)

# Theoretical Pareto and Measured Pareto Overlapped

In order to easily understand how accurate predictions were, an overlapping between the Theoretical Pareto Plot and Measured Pareto Plot was made. In the plot below we have both theoretical (orange) and measured (blue) pareto lines. All measured datapoins are represented as crosses and all theoretical datatpoins are represented as circles. Some theoretical datapoints don't have a measured matched datapoint and the same goes for the measured datapoints. The theoretical pareto curve is, as expected, on the right of the measured one, as predictions are sometimes different form measurements.

In [17]:
#hide_input
%run scripts/overlapped_pareto.py
get_overlapped_pareto('cifar')

alt.LayerChart(...)

# Efficiency Plot

In order to understand the gap between the theoretical predictions and what was measured, and theoretical peak performance, an efficiency bar-chart was created. The size of the bar reflects the absolute performance, whereby all theoretical predictions are shown in red, theoretical peak performance in blue, and all measured datapoints in orange. The orange bars are annotated with the efficiency achieved as a percentage of the predicted performance. Please note the logarithmic y-axis scale. The theoretical predictions take memory bottlenecks into account, as such measured performance can actually exceed the predicted result, in which case the percentage can be above 100%.

In [18]:
#hide_input
%run scripts/overlapped_pareto.py
cifar_efficiency_df = get_peak_perf_gops_df(df_=cnv_df_tmp) #takes the cifar df and fills it with data for the 3rd bar - Theoretical Peak Performance
efficiency_plot(net_keyword= 'cifar', df_theo_peak_compute=cifar_efficiency_df, title='Efficiency plots for CIFAR-10')


alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

In [19]:
#hide
cnv_df.to_csv('data/cleaned_csv/experimental_data_cifar.csv', index = False)
cnv_df.head()

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_datatype_prun,norm-lat-comp,datatype_model,tag
4,CNV,NCS,FP16,NaN,1,6.6,4.9791,151.5550,200.8380,71.1475,94.2834,87.0200,nan,0.5300,1.2000,1.7288,0.4694,100.0000,l3,NCS_FP16_100%,0.0003,FP16_NCS,NCS_FP16_CNV_100.0
5,CNV,NCS,FP16,NaN,2,12.29,8.9383,162.7380,223.7570,76.3974,105.0427,87.0200,nan,0.5300,1.2000,1.7803,0.9389,100.0000,l3,NCS_FP16_100%,0.0006,FP16_NCS,NCS_FP16_CNV_100.0
6,CNV,NCS,FP16,NaN,4,23.35,16.8473,171.2840,237.4260,80.4093,111.4596,87.0200,nan,0.5300,1.2000,1.8230,1.8778,100.0000,l3,NCS_FP16_100%,0.0012,FP16_NCS,NCS_FP16_CNV_100.0
7,CNV,NCS,FP16,NaN,8,45.13,31.8658,177.2590,251.0530,83.2142,117.8568,87.0200,nan,0.5300,1.2000,1.8609,3.7556,100.0000,l3,NCS_FP16_100%,0.0022,FP16_NCS,NCS_FP16_CNV_100.0
8,CNV,NCS,FP16,NaN,16,88.1,61.7732,181.6210,259.0120,85.2620,121.5932,87.0200,nan,0.5300,1.2000,1.8795,7.5112,100.0000,l3,NCS_FP16_100%,0.0042,FP16_NCS,NCS_FP16_CNV_100.0
